In [1]:
import math
import weakref
import random

In [2]:
class Word:
    def __init__(self, wordLength, x, isRegister=True):
        self.wordLength = wordLength
        self.maxValue = 2 ** wordLength - 1
        self.isRegister = isRegister
        
        self._x = x
        self.checkError()
        
    def __del__(self):
        if self.isRegister:
            Machine.deallocateReg(self)
        
    def checkError(self):
        if self._x > self.maxValue or self._x < 0:
            print(f"Error! Tried to set value of {self._x} on the memory with {self.wordLength}-bit word length.")
            assert(False)
            
    def get(self):
        if not self.isRegister:
            Machine.totalExecution += 1
        return self._x
    
    def set(self, x):
        if not self.isRegister:
            Machine.totalExecution += 1
        self._x = x
        self.checkError()
        
    def lg(self):
        v = self.get()
        Machine.totalExecution += 1
        return Machine.allocateReg(int(math.log(v, 2)))
    
    def __add__(self, rhs):
        v = self.get()
        rv = rhs.get()
        Machine.totalExecution += 1
        return Machine.allocateReg(v + rv)
    def __iadd__(self, rhs):
        v = self.get()
        rv = rhs.get()
        Machine.totalExecution += 1
        self.set(v + rv)
        return self
        
    def __sub__(self, rhs):
        v = self.get()
        rv = rhs.get()
        Machine.totalExecution += 1
        return Machine.allocateReg(v + rv)
    def __isub__(self, rhs):
        v = self.get()
        rv = rhs.get()
        Machine.totalExecution += 1
        self.set(v - rv)
        return self
        
    def  __mul__(self, rhs):
        v = self.get()
        rv = rhs.get()
        Machine.totalExecution += 1
        return Machine.allocateReg(v * rv)
    def __imul__(self, rhs):
        v = self.get()
        rv = rhs.get()
        Machine.totalExecution += 1
        self.set(v * rv)
        return self
        
    def __truediv__(self, rhs):
        v = self.get()
        rv = rhs.get()
        Machine.totalExecution += 1
        return Machine.allocateReg(v // rv)
    def __itruediv__(self, rhs):
        v = self.get()
        rv = rhs.get()
        Machine.totalExecution += 1
        self.set(v // rv)
        return self
        
    def __and__(self, rhs):
        v = self.get()
        rv = rhs.get()
        Machine.totalExecution += 1
        return Machine.allocateReg(v & rv)
    def __iand__(self, rhs):
        v = self.get()
        rv = rhs.get()
        Machine.totalExecution += 1
        self.set(r & rv)
        return self
    
    def __or__(self, rhs):
        v = self.get()
        rv = rhs.get()
        Machine.totalExecution += 1
        return Machine.allocateReg(v | rv)
    def __ior__(self, rhs):
        v = self.get()
        rv = rhs.get()
        Machine.totalExecution += 1
        self.set(v | rv)
        return self
    
    def __xor__(self, rhs):
        v = self.get()
        rv = rhs.get()
        Machine.totalExecution += 1
        return Machine.allocateReg(v ^ rv)
    def __ixor__(self, rhs):
        v = self.get()
        rv = rhs.get()
        Machine.totalExecution += 1
        self.set(v ^ rv)
        return self
    
    def __lshift__(self, rhs):
        v = self.get()
        rv = rhs.get()
        Machine.totalExecution += 1
        return Machine.allocateReg(v << rv)
    def __ilshift__(self, rhs):
        v = self.get()
        rv = rhs.get()
        Machine.totalExecution += 1
        self.set(v << rv)
        return self
    
    def __rshift__(self, rhs):
        v = self.get()
        rv = rhs.get()
        Machine.totalExecution += 1
        return Machine.allocateReg(v << rv)
    def __irshift__(self, rhs):
        v = self.get()
        rv = rhs.get()
        Machine.totalExecution += 1
        self.set(v >> rv)
        return self
    
    def _randomize(self):
        self._x = random.randint(0, self.maxValue)
        
    def _getBit(self, i):
        return (self._x >> (self.wordLength - i - 1)) & 1

In [3]:
class WordRAM:
    def __init__(self, size, wordLength):
        self.size = size
        self.wordLength = wordLength
        
        self._d = [Word(self.wordLength, 0, False) for _ in range(self.size)]
    
    def __getitem__(self, i):
        assert(isinstance(i, Word))
        x = i.get()
        assert(x < self.size)
        return self._d[x]
    
    def _randomize(self):
        for word in self._d:
            word._randomize()
            
    def _getBit(self, i):
        blockId = i // self.wordLength
        bitIdInBlock = i - blockId * self.wordLength
        return self._d[blockId]._getBit(bitIdInBlock)

In [4]:
class WordRAMMachine:
    def __init__(self, wordLength, registerNum):
        self.wordLength = wordLength
        
        self.registerNum = registerNum
        self.crtRegisters = weakref.WeakSet()
        
        self.totalMemSize = 0
        self.totalExecution = 0
        
    def allocateMem(self, size, wordLength):
        size = size.get()
        wordLength = wordLength.get()
        if (wordLength > self.wordLength):
            print(f"Error! Tried to allocate memory with word length of {wordLength} bits on the machine with {self.wordLength}-bit word length.")
            return None
        mem = WordRAM(size, wordLength)
        self.totalMemSize += size * wordLength
        return mem
    
    def allocateReg(self, x=0):
        assert(len(self.crtRegisters) < self.registerNum)
        reg = Word(self.wordLength, x)
        self.crtRegisters.add(reg)
        return reg
    
    def deallocateReg(self, reg):
        self.crtRegisters.remove(reg)

In [5]:
Machine = None

def CreateMachine(wordLength, registerNum):
    global Machine
    assert(Machine is None)
    Machine = WordRAMMachine(wordLength, registerNum)
    
def Reg(x):
    return Machine.allocateReg(x)

def Mem(size, wordLength):
    return Machine.allocateMem(size, wordLength)